In [7]:
import pandas as pd
column_names = [
    'Wikipedia article ID',
    'Freebase ID',
    'Book title',
    'Author',
    'Publication date',
    'Book genres (Freebase ID:name tuples)',
    'Plot summary'
]
df = pd.read_csv('data/booksummaries.txt', delimiter = '\t', names = column_names, header = None)


In [8]:
df.head()

,Wikipedia article ID,Freebase ID,Book title,Author,Publication date,Book genres (Freebase ID:name tuples),Plot summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [9]:
# import os
# import pandas as pd
# from openai import OpenAI
# from dotenv import load_dotenv
# from tqdm import tqdm
# import time

# # 환경 변수 로드 및 OpenAI 클라이언트 설정
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# client = OpenAI(api_key=OPENAI_API_KEY)
# # 진행률 표시 설정
# tqdm.pandas()

# # 요약 함수 (OpenAI 최신 방식 적용)
# def summarize_text(text, max_tokens=500):
#     prompt = f"Summarize the following book plot in less than 500 characters:\n\n{text}"
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": "You are a helpful assistant."},
#                 {"role": "user", "content": prompt}
#             ],
#             max_tokens=max_tokens,
#             temperature=0.5  # 출력의 다양성 조절
#         )
#         answer = response.choices[0].message.content.strip()
#         return answer
#     except Exception as e:
#         print(f"Error: {e}")
#         return None

# # 요약 적용 함수 (결측치 제외)
# def apply_summarization(text):
#     if pd.notnull(text) and text.strip() != "":
#         return summarize_text(text)
#     return ""

# df_sample = df.head(10)

# # 요약 컬럼 추가 (진행률 표시)
# df_sample['Summarized Plot Summary'] = df_sample['Plot summary'].progress_apply(apply_summarization)

# # 결과 확인
# print(df_sample[['Book title', 'Summarized Plot Summary']].head())

In [10]:
# df_sample[['Book title', 'Summarized Plot Summary']].to_csv("summarized_plots.csv", index=False, encoding='utf-8-sig')
# print("결과가 summarized_plots.csv 파일로 저장되었습니다.")


In [11]:
df.shape

(16559, 7)

In [12]:
# Plot summary와 Book genres 컬럼에서 결측치(NaN) 또는 빈 값("")인 행 삭제
df_cleaned = df.dropna(subset=['Plot summary', 'Book genres (Freebase ID:name tuples)'])

# 추가로 공백("")인 경우도 제거
df_cleaned = df_cleaned[
    (df_cleaned['Plot summary'].str.strip() != '') &
    (df_cleaned['Book genres (Freebase ID:name tuples)'].str.strip() != '')
]
print(f"before {df.shape}")
# 결과 확인
print(df_cleaned[['Book title', 'Plot summary', 'Book genres (Freebase ID:name tuples)']].head())
print(f"전처리 후 데이터 크기: {df_cleaned.shape}")


before (16559, 7)
                       Book title  \
0                     Animal Farm   
1              A Clockwork Orange   
2                      The Plague   
4            A Fire Upon the Deep   
5  All Quiet on the Western Front   

                                        Plot summary  \
0   Old Major, the old boar on the Manor Farm, ca...   
1   Alex, a teenager living in near-future Englan...   
2   The text of The Plague is divided into five p...   
4   The novel posits that space around the Milky ...   
5   The book tells the story of Paul Bäumer, a Ge...   

               Book genres (Freebase ID:name tuples)  
0  {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...  
1  {"/m/06n90": "Science Fiction", "/m/0l67h": "N...  
2  {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...  
4  {"/m/03lrw": "Hard science fiction", "/m/06n90...  
5  {"/m/098tmk": "War novel", "/m/016lj8": "Roman...  
전처리 후 데이터 크기: (12841, 7)


In [13]:
# Book title 컬럼에서 중복 제거
df_cleaned = df_cleaned.drop_duplicates(subset='Book title', keep='first')

# 결과 확인
print(df_cleaned[['Book title', 'Author']].head())
print(f"중복 제거 후 데이터 크기: {df_cleaned.shape}")


                       Book title                Author
0                     Animal Farm         George Orwell
1              A Clockwork Orange       Anthony Burgess
2                      The Plague          Albert Camus
4            A Fire Upon the Deep          Vernor Vinge
5  All Quiet on the Western Front  Erich Maria Remarque
중복 제거 후 데이터 크기: (12637, 7)


In [14]:
# import os
# import pandas as pd
# from openai import OpenAI
# from dotenv import load_dotenv
# from tqdm import tqdm
# import time

# #  환경 변수 로드 및 OpenAI 클라이언트 설정
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# client = OpenAI(api_key = OPENAI_API_KEY)
# # 진행률 표시 설정
# tqdm.pandas()

# # 요약 함수
# def summarize_text(text, max_tokens=500):
#     prompt = f"Summarize the following book plot in less than 500 characters:\n\n{text}"
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": "You are a helpful assistant."},
#                 {"role": "user", "content": prompt}
#             ],
#             max_tokens=max_tokens,
#             temperature=0.5
#         )
#         answer = response.choices[0].message.content.strip()
#         return answer
#     except Exception as e:
#         print(f"Error: {e}")
#         return None

# # 요약 적용 함수
# def apply_summarization(text):
#     if pd.notnull(text) and text.strip() != "":
#         return summarize_text(text)
#     return ""

# #  처리 범위 설정
# start_idx = 0     # 시작 인덱스 (수동 입력)
# end_idx = 3000    # 끝 인덱스 (수동 입력)

# #  데이터프레임 슬라이싱 (범위에 맞게 분할)
# df_chunk = df.iloc[start_idx:end_idx].copy()

# print(f"\n Processing {start_idx} ~ {end_idx} rows...")

# #  요약 컬럼 추가
# df_chunk['Summarized Plot Summary'] = df_chunk['Plot summary'].progress_apply(apply_summarization)

# #  결과 저장
# save_path = f'/data/booksummaries_summarized_{start_idx}_{end_idx}.csv'
# df_chunk.to_csv(save_path, index=False)
# print(f" 요약 완료! 파일 저장 경로: {save_path}")


In [15]:
# #  결과 저장
# save_path = f'data/booksummaries_summarized_{start_idx}_{end_idx}.csv'
# df_chunk.to_csv(save_path, index=False)
# print(f" 요약 완료! 파일 저장 경로: {save_path}")

In [16]:
# # 요약본의 평균 글자수를 계산
# average_length = df_chunk['Summarized Plot Summary'].apply(len).mean()
# average_length

In [17]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import time

#  환경 변수 로드 및 OpenAI 클라이언트 설정
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key = OPENAI_API_KEY)
# 진행률 표시 설정
tqdm.pandas()

# 요약 함수
def summarize_text(text, max_tokens=500):
    prompt = f"Summarize the following book plot in less than 500 characters:\n\n{text}"
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.5
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return None

# 요약 적용 함수
def apply_summarization(text):
    if pd.notnull(text) and text.strip() != "":
        return summarize_text(text)
    return ""

#  처리 범위 설정
start_idx = 3001     # 시작 인덱스 (수동 입력)
end_idx = 9000  # 끝 인덱스 (수동 입력)

#  데이터프레임 슬라이싱 (범위에 맞게 분할)
df_chunk = df.iloc[start_idx:end_idx].copy()

print(f"\n Processing {start_idx} ~ {end_idx} rows...")

#  요약 컬럼 추가
df_chunk['Summarized Plot Summary'] = df_chunk['Plot summary'].progress_apply(apply_summarization)

#  결과 저장
save_path = f'data/booksummaries_summarized_{start_idx}_{end_idx}.csv'
df_chunk.to_csv(save_path, index=False)
print(f" 요약 완료! 파일 저장 경로: {save_path}")



 Processing 3001 ~ 9000 rows...


100%|██████████| 5999/5999 [4:51:23<00:00,  2.91s/it]   


 요약 완료! 파일 저장 경로: data/booksummaries_summarized_3001_9000.csv


In [19]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import time

#  환경 변수 로드 및 OpenAI 클라이언트 설정
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key = OPENAI_API_KEY)
# 진행률 표시 설정
tqdm.pandas()

# 요약 함수
def summarize_text(text, max_tokens=500):
    prompt = f"Summarize the following book plot in less than 500 characters:\n\n{text}"
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.5
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return None

# 요약 적용 함수
def apply_summarization(text):
    if pd.notnull(text) and text.strip() != "":
        return summarize_text(text)
    return ""

#  처리 범위 설정
start_idx = 9001     # 시작 인덱스 (수동 입력)
end_idx = len(df)  # 끝 인덱스 (수동 입력)

#  데이터프레임 슬라이싱 (범위에 맞게 분할)
df_chunk = df.iloc[start_idx:end_idx].copy()

print(f"\n Processing {start_idx} ~ {end_idx} rows...")

#  요약 컬럼 추가
df_chunk['Summarized Plot Summary'] = df_chunk['Plot summary'].progress_apply(apply_summarization)

#  결과 저장
save_path = f'data/booksummaries_summarized_{start_idx}_{end_idx}.csv'
df_chunk.to_csv(save_path, index=False)
print(f" 요약 완료! 파일 저장 경로: {save_path}")



 Processing 9001 ~ 16559 rows...


100%|██████████| 7558/7558 [5:09:24<00:00,  2.46s/it]   


 요약 완료! 파일 저장 경로: data/booksummaries_summarized_9001_16559.csv
